In [1]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [ ]:
"""MCP client."""
from contextlib import asynccontextmanager, AsyncExitStack
import asyncio

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv

load_dotenv()


SERVER_CONFIGS = {
    "playwright": {
        "url": "http://localhost:8931/mcp",
        "transport": "streamable_http",
    },
    "notion":{
        "url": "https://server.smithery.ai/@smithery/notion/mcp?profile=scary-turtle-Ljf5sY&api_key=a457b5a4-cd03-4a13-b2ac-cf99c04f6fc4",
        "transport": "streamable_http",
    },
}

mcp_client = MultiServerMCPClient(SERVER_CONFIGS)

async with AsyncExitStack() as st:
    server_names = SERVER_CONFIGS.keys()
    sessions = [
        await st.enter_async_context(mcp_client.session(server))
        for server in server_names
    ]
    tools = sum(await asyncio.gather(*map(load_mcp_tools, sessions)), [])

    agent = create_react_agent(
        "openai:gpt-4.1",
        tools,
    )

    while True:
        input_text = input("\n[USER] ")
        if input_text.strip().lower() in ["종료", "exit"]:
            print("대화를 종료합니다.")
            break

        result = await agent.ainvoke({"messages": HumanMessage(content=input_text)})
        output = result['messages'][-1].pretty_print()    